# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [104]:
# Import Python packages +++
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [106]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [107]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [108]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. The Apache Cassandra coding portion of project. 

## Required dataset for Sparkify mobile app events is contained in the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Connecting to Apache Cassandra cluster and creating Sparkify keyspace

#### Creating a Cluster

In [109]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])

    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [110]:
# Create a Keyspace 
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify
        WITH REPLICATION = 
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [111]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### -----------------------------------------------------------------------------------------------------------------------------------------------------------

### Query 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [112]:
# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
# sessionId = 338, and itemInSession = 4
drop_query = "DROP TABLE IF EXISTS session_song_library"
create_query = "CREATE TABLE IF NOT EXISTS session_song_library "
create_query = create_query + "(sessionId int, itemInSession int, artist text, song text, length float, \
                                PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(drop_query)
    session.execute(create_query)
except Exception as e:
    print(e)                    

In [113]:
# Insert data from event_datafile_new.csv into session_song_library table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    # Execute INSERT statement for every line from csv file 
        query = "INSERT INTO session_song_library (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
                print(e)

#### Do a SELECT to verify that correct answer is returned for given question

In [114]:
# Validate data model
# Execute SELECT statement for Query #1
query = "SELECT artist, song, length FROM session_song_library WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row[0], row[1], round(row[2],5))

Faithless Music Matters (Mark Knight Dub) 495.30731


### Query 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [115]:
# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
# for userid = 10, sessionid = 182
drop_query = "DROP TABLE IF EXISTS session_library"
create_query = "CREATE TABLE IF NOT EXISTS session_library "
create_query = create_query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, \
                                lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(drop_query)
    session.execute(create_query)
except Exception as e:
    print(e)                    

In [116]:
# Insert data from event_datafile_new.csv into session_library table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    # Execute INSERT statement for every line from csv file 
        query = "INSERT INTO session_library (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        except Exception as e:
                print(e)

#### Do a SELECT to verify that correct answer is returned for given question

In [117]:
# Validate data model
# Execute SELECT statement for Query #2
query = 'SELECT artist, itemInSession, song, firstName as "userFirstName", lastName as "userLastName"\
        FROM session_library WHERE userId = 10 AND sessionId = 182'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row[0], row[1], row[2], row[3], row[4])

Down To The Bone 0 Keep On Keepin' On Sylvie Cruz
Three Drives 1 Greece 2000 Sylvie Cruz
Sebastien Tellier 2 Kilometer Sylvie Cruz
Lonnie Gordon 3 Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [118]:
# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands\
# Against His Own'
drop_query = "DROP TABLE IF EXISTS song_history_library"
create_query = "CREATE TABLE IF NOT EXISTS song_history_library "
create_query = create_query + "(song text, userId int, firstName text, lastName text,\
                                PRIMARY KEY (song, userId))"
try:
    session.execute(drop_query)
    session.execute(create_query)
except Exception as e:
    print(e)                    

In [119]:
# Insert data from event_datafile_new.csv into session_library table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    # Execute INSERT statement for every line from csv file 
        query = "INSERT INTO song_history_library (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        try:
            session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        except Exception as e:
                print(e)

#### Do a SELECT to verify that correct answer is returned for given question

In [120]:
# Validate data model
# Execute SELECT statement for Query #3
query = """SELECT song, firstName as "userFirstName", lastName as "userLastName"
            FROM song_history_library WHERE song = 'All Hands Against His Own'"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row[0], row[1], row[2])

All Hands Against His Own Jacqueline Lynch
All Hands Against His Own Tegan Levine
All Hands Against His Own Sara Johnson


### Drop the tables before closing out the sessions

In [121]:
# Drop session_song_library table before closing out the sessions
query = "DROP TABLE session_song_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [122]:
# Drop session_library table before closing out the sessions
query = "DROP TABLE session_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [123]:
# Drop session_library table before closing out the sessions
query = "DROP TABLE song_history_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [124]:
session.shutdown()
cluster.shutdown()